# **Задача 2.**

Второй датасет - про покупателей велосипедов.

Пытаемся по характеристикам человека понять, купит он велик или нет.


1. Marital_Status	- Семейное положение
2. Gender	- пол
3. Income	- доход
4. Children	- сколько детей
5. Education	- уровень образования
6. Occupation	- вид занятости
7. Home_Owner	- наличие частного дома
8. Cars	- наличие машины
9. Commute_Distance	- Расстояние до места работы
10. Region - регион проживания
11. Age	- возраст
12. Purchased_Bike - покупка велосипеда



# 1. Импортируем нужные библиотеки

In [144]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# 2. Загружаем наш датасет

In [145]:
bike_df = pd.read_csv('bike_buyers_clean.csv')
bike_df.head()

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


# 3. Оценка данных

In [146]:
# выведем размерность нашего датасета:
bike_df.shape

(1000, 13)

In [147]:
# выведем типы данных:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                1000 non-null   int64 
 1   Marital Status    1000 non-null   object
 2   Gender            1000 non-null   object
 3   Income            1000 non-null   int64 
 4   Children          1000 non-null   int64 
 5   Education         1000 non-null   object
 6   Occupation        1000 non-null   object
 7   Home Owner        1000 non-null   object
 8   Cars              1000 non-null   int64 
 9   Commute Distance  1000 non-null   object
 10  Region            1000 non-null   object
 11  Age               1000 non-null   int64 
 12  Purchased Bike    1000 non-null   object
dtypes: int64(5), object(8)
memory usage: 101.7+ KB


In [148]:
# выведем основные статистические показатели:
bike_df.describe().round(3)

,ID,Income,Children,Cars,Age
count,1000.000,1000.00,1000.000,1000.000,1000.000
mean,19965.992,56140.00,1.908,1.452,44.190
std,5347.334,31081.61,1.626,1.125,11.354
min,11000.000,10000.00,0.000,0.000,25.000
25%,15290.750,30000.00,0.000,1.000,35.000
50%,19744.000,60000.00,2.000,1.000,43.000
75%,24470.750,70000.00,3.000,2.000,52.000
max,29447.000,170000.00,5.000,4.000,89.000


# 4. Feature Engineering:

## 4.1. Удалим ненужные столбцы

In [149]:
# Удалим столбец ID:
bike_df.drop(['ID'], axis=1, inplace=True)
bike_df

,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Married,Male,60000,2,High School,Professional,Yes,2,2-5 Miles,North America,54,Yes
996,Single,Male,70000,4,Graduate Degree,Professional,Yes,0,2-5 Miles,North America,35,Yes
997,Married,Male,60000,2,Bachelors,Skilled Manual,Yes,0,0-1 Miles,North America,38,Yes
998,Single,Male,100000,3,Bachelors,Management,No,3,1-2 Miles,North America,38,No


## 4.2. Переименуем столбцы

In [150]:
def Rename(data, feature1, feature2):
  data.rename(columns={feature1: feature2}, inplace=True)
  return(data)

In [151]:
Rename(bike_df, 'Marital Status', 'Marital_Status')
Rename(bike_df, 'Home Owner', 'Home_Owner')
Rename(bike_df, 'Commute Distance', 'Commute_Distance')
Rename(bike_df, 'Purchased Bike', 'Purchased_Bike')
bike_df

,Marital_Status,Gender,Income,Children,Education,Occupation,Home_Owner,Cars,Commute_Distance,Region,Age,Purchased_Bike
0,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Married,Male,60000,2,High School,Professional,Yes,2,2-5 Miles,North America,54,Yes
996,Single,Male,70000,4,Graduate Degree,Professional,Yes,0,2-5 Miles,North America,35,Yes
997,Married,Male,60000,2,Bachelors,Skilled Manual,Yes,0,0-1 Miles,North America,38,Yes
998,Single,Male,100000,3,Bachelors,Management,No,3,1-2 Miles,North America,38,No


## 4.3. Обработка категориальных признаков и создание новых признаков из уже существующих

In [152]:
print('Marital_Status: ', bike_df.Marital_Status.unique(), '\n',
      'Gender: ', bike_df.Gender.unique(), '\n',
      'Income: ', sorted(bike_df.Income.unique()), '\n',
      'Education: ', bike_df.Education.unique(), '\n',
      'Occupation: ', bike_df.Occupation.unique(), '\n',
      'Home_Owner: ', bike_df.Home_Owner.unique(), '\n',
      'Commute_Distance: ', bike_df.Commute_Distance.unique(), '\n',
      'Region: ', bike_df.Region.unique(), '\n',
      'Purchased_Bike: ', bike_df.Purchased_Bike.unique()
      )

Marital_Status:  ['Married' 'Single'] 
 Gender:  ['Female' 'Male'] 
 Income:  [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 150000, 160000, 170000] 
 Education:  ['Bachelors' 'Partial College' 'High School' 'Partial High School'
 'Graduate Degree'] 
 Occupation:  ['Skilled Manual' 'Clerical' 'Professional' 'Manual' 'Management'] 
 Home_Owner:  ['Yes' 'No'] 
 Commute_Distance:  ['0-1 Miles' '2-5 Miles' '5-10 Miles' '1-2 Miles' '10+ Miles'] 
 Region:  ['Europe' 'Pacific' 'North America'] 
 Purchased_Bike:  ['No' 'Yes']


In [153]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

def LabelEncoder (data, feature):
  data[feature] = label_encoder.fit_transform(data[feature])

  return(data)

In [154]:
from sklearn.preprocessing import OneHotEncoder

def One_Hot_Encoder (data, feature):
  encoded_columns = pd.get_dummies(data[feature])
  return(encoded_columns)

In [155]:
def change_type(data, feature):
  data[feature] = data[feature].astype(int)
  return(data)

In [156]:
# Поработаем с признаком 'Marital_Status' (Семейное положение):
bike_df.Marital_Status.replace(['Single', 'Married'], [0, 1], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'Gender' (пол):
bike_df = bike_df.join(One_Hot_Encoder(bike_df, 'Gender')) # применим кодирование One_Hot_Encoder
change_type(bike_df, 'Female')
change_type(bike_df, 'Male')


# Поработаем с признаком 'Income' (доход):
def map_income(income):
  if income == '10000-50000':
    return 0
  elif income == '60000-100000':
    return 1
  elif income == '110000-150000':
    return 2
  else:
    return 3
bike_df['Income'] = bike_df['Income'].apply(map_income)


# Поработаем с признаком 'Education' (образование):
LabelEncoder(bike_df, 'Education') # применим кодирование LabelEncoder


# Поработаем с признаком 'Occupation' (вид занятости):
LabelEncoder(bike_df, 'Occupation') # применим кодирование LabelEncoder


# Поработаем с признаком 'Home_Owner' (наличие частного дома):
bike_df.Home_Owner.replace(['No', 'Yes'], [0, 1], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'Commute_Distance' (наличие частного дома):
bike_df.Commute_Distance.replace(['0-1 Miles','2-5 Miles', '5-10 Miles', '1-2 Miles', '10+ Miles'], [0, 1, 2, 3, 4], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'Age' (возраст):
def map_age(age):
  if age == '0-17':
    return 0
  elif age == '18-25':
    return 1
  elif age == '26-35':
    return 2
  elif age == '36-45':
    return 3
  elif age == '46-50':
    return 4
  elif age == '51-55':
    return 5
  else:
    return 6
bike_df['Age'] = bike_df['Age'].apply(map_age)


# Поработаем с признаком 'Region' (регион проживания):
bike_df = bike_df.join(One_Hot_Encoder(bike_df, 'Region')) # применим кодирование One_Hot_Encoder
change_type(bike_df, 'Europe')
change_type(bike_df, 'North America')
change_type(bike_df, 'Pacific')


# Поработаем с признаком 'Purchased_Bike' (наличие велосипеда):
bike_df.Purchased_Bike.replace(['No', 'Yes'], [0, 1], inplace=True) # применим кодирование через метод replace()



drop_elements = ['Gender', 'Region']

bike_df = bike_df.drop(drop_elements, axis = 1)

bike_df

,Marital_Status,Income,Children,Education,Occupation,Home_Owner,Cars,Commute_Distance,Age,Purchased_Bike,Female,Male,Europe,North America,Pacific
0,1,3,1,0,4,1,0,0,6,0,1,0,1,0,0
1,1,3,3,3,0,1,1,0,6,0,0,1,1,0,0
2,1,3,5,3,3,0,2,1,6,0,0,1,1,0,0
3,0,3,0,0,3,1,1,2,6,1,0,1,0,0,1
4,0,3,0,0,0,0,0,0,6,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,3,2,2,3,1,2,1,6,1,0,1,0,1,0
996,0,3,4,1,3,1,0,1,6,1,0,1,0,1,0
997,1,3,2,0,4,1,0,0,6,1,0,1,0,1,0
998,0,3,3,0,1,0,3,3,6,0,0,1,0,1,0


In [157]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Marital_Status    1000 non-null   int64
 1   Income            1000 non-null   int64
 2   Children          1000 non-null   int64
 3   Education         1000 non-null   int64
 4   Occupation        1000 non-null   int64
 5   Home_Owner        1000 non-null   int64
 6   Cars              1000 non-null   int64
 7   Commute_Distance  1000 non-null   int64
 8   Age               1000 non-null   int64
 9   Purchased_Bike    1000 non-null   int64
 10  Female            1000 non-null   int64
 11  Male              1000 non-null   int64
 12  Europe            1000 non-null   int64
 13  North America     1000 non-null   int64
 14  Pacific           1000 non-null   int64
dtypes: int64(15)
memory usage: 117.3 KB


# 5. Построим модель Логистической регрессии

In [158]:
X = bike_df.drop('Purchased_Bike', axis=1)
y = bike_df.Purchased_Bike
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [159]:
# Проверим распределение классов:
y.value_counts()

0    519
1    481
Name: Purchased_Bike, dtype: int64

In [160]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.61      0.63      0.62       365
           1       0.64      0.62      0.63       385

    accuracy                           0.63       750
   macro avg       0.63      0.63      0.63       750
weighted avg       0.63      0.63      0.63       750
               precision    recall  f1-score   support

           0       0.63      0.67      0.65       132
           1       0.60      0.55      0.58       118

    accuracy                           0.62       250
   macro avg       0.61      0.61      0.61       250
weighted avg       0.62      0.62      0.61       250



In [161]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.64      0.63      0.63       387
           1       0.61      0.63      0.62       363

    accuracy                           0.63       750
   macro avg       0.63      0.63      0.63       750
weighted avg       0.63      0.63      0.63       750
               precision    recall  f1-score   support

           0       0.64      0.67      0.65       136
           1       0.58      0.55      0.57       114

    accuracy                           0.62       250
   macro avg       0.61      0.61      0.61       250
weighted avg       0.61      0.62      0.61       250

